# Race/ethnicity by year

## Setup

### Packages

In [78]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

In [79]:
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: C:\Users\ogwynn\OneDrive - Mid-Ohio Regional Planning Commission\Desktop\LocalRepo\makingInsights


### Paths

In [80]:
INPUT_DATA_PATH = "inputData/censusPep_county_char.csv"

TEMP_OUTPUT_CSV_PATH ="outputData/TEMP.csv"

REGIONAL_SUMMARY_DATA_PATH = "outputData/censusPep_county_char_15Region.csv"

REGIONAL_SUMMARY_DATA_LABELED_PATH = "outputData/censusPep_county_char_labeled_15Region.csv"

data1 = pd.read_csv(INPUT_DATA_PATH)

data1.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

data1.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
0,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2000,1
1,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2001,1
2,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2002,1
3,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2003,1
4,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2004,1


## Filtering

### Filtering for 15 county

In [81]:
# List of counties to filter by
selected_counties = [
    "Delaware", "Fairfield", "Fayette", "Franklin", "Hocking", "Knox", "Licking", 
    "Logan", "Madison", "Marion", "Morrow", "Perry", "Pickaway", "Ross", "Union"
]

selected_years = [
    2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023
]

# Filter the dataset for the selected counties
data2 = data1[data1['COUNTY'].isin(selected_counties)]

# Filter the dataset for the selected counties
data2 = data2[data2['YEAR'].isin(selected_years)]

# Filter out rows where 'SOURCE' is 'INT' and 'YEAR' is '2010'
data2 = data2[~((data2['SOURCE'] == 'INT') & (data2['YEAR'] == '2010'))]

data2.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

# Display the first few rows of the filtered data
data2.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
178210,39041,Ohio,Delaware,F,AA,0 to 4,INT,2012,2010,389
178221,39041,Ohio,Delaware,F,AA,10 to 14,INT,2012,2010,274
178232,39041,Ohio,Delaware,F,AA,15 to 19,INT,2012,2010,187
178243,39041,Ohio,Delaware,F,AA,20 to 24,INT,2012,2010,139
178254,39041,Ohio,Delaware,F,AA,25 to 29,INT,2012,2010,278


### Summing by 15 County

In [82]:
# Group by the relevant columns and sum the 'POP' variable
data2_1 = data2.groupby(
    ['SEX', 'RACE', 'AGEGROUP', 'YEAR']
)['POP'].sum().reset_index()

# Add a new column for the COUNTY identifier
data2_1['COUNTY'] = '15County'

# Rearrange columns to match the original dataset order
data2_1 = data2_1[['COUNTY', 'SEX', 'RACE', 'AGEGROUP','YEAR', 'POP']]

# Append the new '15County' entries to the filtered dataset
data3 = pd.concat([data2, data2_1], ignore_index=True)

data3.to_csv(REGIONAL_SUMMARY_DATA_PATH, index=False)

# Display the first few rows of the updated data
data3.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
0,39041.0,Ohio,Delaware,F,AA,0 to 4,INT,2012.0,2010,389
1,39041.0,Ohio,Delaware,F,AA,10 to 14,INT,2012.0,2010,274
2,39041.0,Ohio,Delaware,F,AA,15 to 19,INT,2012.0,2010,187
3,39041.0,Ohio,Delaware,F,AA,20 to 24,INT,2012.0,2010,139
4,39041.0,Ohio,Delaware,F,AA,25 to 29,INT,2012.0,2010,278


### Filtering for sex totals and summing for different races

In [83]:
# Filter the dataset where SEX == 'TOT'
data3_1 = data3[data3['SEX'] != 'TOT']

# Group by the relevant columns and sum the 'POP' variable
data4 = data3_1.groupby(
    ['COUNTY', 'RACE', 'YEAR']
)['POP'].sum().reset_index()

data4.to_csv(REGIONAL_SUMMARY_DATA_PATH, index=False)

# Display the first few rows of the updated data
data4.head()

,COUNTY,RACE,YEAR,POP
0,15County,AA,2010,121054
1,15County,AA,2011,63769
2,15County,AA,2012,66927
3,15County,AA,2013,71774
4,15County,AA,2014,77219


### Combining RACE NA

In [84]:
# Step 1: Filter data for RACE == 'HNA' and RACE == 'NHNA'
hna_nhna_data = data4[data4['RACE'].isin(['HNA', 'NHNA'])]

# Step 2: Group by COUNTY, SOURCE, VINTAGE, and YEAR, and sum the POP
combined_pop = hna_nhna_data.groupby(['COUNTY', 'YEAR'], as_index=False)['POP'].sum()

# Step 3: Add a new column for RACE with value 'NA'
combined_pop['RACE'] = 'NA'


# Step 4: Append the new rows to the original data
data5 = pd.concat([data4, combined_pop], ignore_index=True)

data5.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

### Combining hispanic 

In [85]:
# Step 1: Filter data for RACE == 'HNA' and RACE == 'NHNA'
h_data = data5[data5['RACE'].isin(['HNA', 'HBA', 'HAA', 'HIA', 'HWA', 'HTOM'])]

# Step 2: Group by COUNTY, SOURCE, VINTAGE, and YEAR, and sum the POP
combined_pop = h_data.groupby(['COUNTY', 'YEAR'], as_index=False)['POP'].sum()

# Step 3: Add a new column for RACE with value 'NA'
combined_pop['RACE'] = 'HH'

# Step 4: Append the new rows to the original data
data6 = pd.concat([data5, combined_pop], ignore_index=True)

data6.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

### Combining other races (NHIA + NHNA)

In [86]:
# Step 1: Filter data for RACE == 'HNA' and RACE == 'NHNA'
other_data = data6[data6['RACE'].isin(['NHIA', 'NHNA'])]

# Step 2: Group by COUNTY, SOURCE, VINTAGE, and YEAR, and sum the POP
combined_pop = other_data.groupby(['COUNTY', 'YEAR'], as_index=False)['POP'].sum()

# Step 3: Add a new column for RACE with value 'NA'
combined_pop['RACE'] = 'OTHER'

# Step 4: Append the new rows to the original data
data7 = pd.concat([data6, combined_pop], ignore_index=True)

data7.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

# Display the first few rows of the updated dataframe
data7.head()

,COUNTY,RACE,YEAR,POP
0,15County,AA,2010,121054
1,15County,AA,2011,63769
2,15County,AA,2012,66927
3,15County,AA,2013,71774
4,15County,AA,2014,77219


### Proportions

In [87]:
# Filter the dataset to include only the specified RACE categories, including 'TOT'
race_categories = {'NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER', 'TOT'}
filtered_data = data7[data7['RACE'].isin(race_categories)]

# Pivot data to have RACE as columns for easier calculation of proportions
pivot_df = filtered_data.pivot_table(index=['COUNTY', 'YEAR'], columns='RACE', values='POP', fill_value=0)

# Calculate proportions by dividing each race population by the total population (TOT)
for race in ['NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER']:
    pivot_df[f'{race}_proportion'] = pivot_df[race] / pivot_df['TOT']

# Reset index for a more conventional DataFrame format
proportions_df = pivot_df.reset_index()

# Display the resulting DataFrame
proportions_df[['COUNTY', 'YEAR'] + [f'{race}_proportion' for race in ['NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER']]]


df=proportions_df


pop_df = df.melt(id_vars=['COUNTY', 'YEAR'], value_vars=['HH', 'NHAA', 'NHBA', 'NHTOM', 'NHWA', 'OTHER'], 
                 var_name='RACE', value_name='POP')

# Melt the proportion columns to get 'POP_PROP' values for each RACE
prop_df = df.melt(id_vars=['COUNTY', 'YEAR'], 
                  value_vars=['HH_proportion', 'NHAA_proportion', 'NHBA_proportion', 'NHTOM_proportion', 'NHWA_proportion', 'OTHER_proportion'], 
                  var_name='RACE_PROP', value_name='POP_PROP')

# Adjust RACE column in the proportion data to match the format in pop_df
prop_df['RACE'] = prop_df['RACE_PROP'].str.replace('_proportion', '')

# Merge the two DataFrames on COUNTY, YEAR, and RACE to get POP and POP_PROP in a single DataFrame
data8 = pd.merge(pop_df, prop_df[['COUNTY', 'YEAR', 'RACE', 'POP_PROP']], on=['COUNTY', 'YEAR', 'RACE'])

data8.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

data8.head()

,COUNTY,YEAR,RACE,POP,POP_PROP
0,15County,2010,HH,142849.0,0.032664
1,15County,2011,HH,73501.0,0.033310
2,15County,2012,HH,75822.0,0.034059
3,15County,2013,HH,78700.0,0.034933
4,15County,2014,HH,81701.0,0.035826


### Label map

In [88]:
# Now, filter out records where RACE is NOT in the specified set
filtered_data = data8[data8['RACE'].isin(['NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER'])]

# Define a comprehensive mapping of race abbreviations to full race names
race_mapping = {
    "NHAA": "Asian",
    "HH": "Hispanic or Latio (any race)",
    "NHBA": "Black or African American",
    "OTHER": "Some other race alone",
    "NHTOM": "Two or more races",
    "NHWA": "White"
}

# Apply the mapping to the 'RACE' column
filtered_data['RACE'] = filtered_data['RACE'].map(race_mapping).fillna(filtered_data['RACE'])

filtered_data.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

# Display the first few rows of the updated data
filtered_data.head()

,COUNTY,YEAR,RACE,POP,POP_PROP
0,15County,2010,Hispanic or Latio (any race),142849.0,0.032664
1,15County,2011,Hispanic or Latio (any race),73501.0,0.033310
2,15County,2012,Hispanic or Latio (any race),75822.0,0.034059
3,15County,2013,Hispanic or Latio (any race),78700.0,0.034933
4,15County,2014,Hispanic or Latio (any race),81701.0,0.035826


## Garphing

### Creating images for each region

In [102]:
import matplotlib.pyplot as plt

# Load the uploaded file to review its content
file_path =REGIONAL_SUMMARY_DATA_LABELED_PATH

# Load the data from the CSV file
data = pd.read_csv(file_path)

# Set up output directory
output_dir = 'outputData/plots'

os.makedirs(output_dir, exist_ok=True)

# Dictionary to store COUNTY to itemID mapping
county_item_mapping = {}

# Iterate over each county
for county in data['COUNTY'].unique():
    county_data = data[data['COUNTY'] == county]
    
    # Ensure 'YEAR', 'POP_PROP', and 'RACE' columns are present
    county_data = county_data[['YEAR', 'POP_PROP', 'RACE']]
    
    # Aggregate to handle duplicates by taking the mean
    county_data = county_data.groupby(['YEAR', 'RACE'], as_index=False).agg({'POP_PROP': 'mean'})
    
    # Pivot data to have 'RACE' as columns for plotting
    county_data_pivot = county_data.pivot(index='YEAR', columns='RACE', values='POP_PROP')
    
    # Plot each county's data with adjacent bars
    plt.figure(figsize=(16, 9))
    bar_width = 0.15  # Reduced width for better spacing
    x_positions = range(len(county_data_pivot.index))
    
    # Improved color palette
    colors = ['#365072', '#34617A', '#0075BF', '#00B2BF', '#2A7F67', '#5AB65F']
    
    for i, (race, color) in enumerate(zip(county_data_pivot.columns, colors)):
        plt.bar(
            [x + i * bar_width for x in x_positions],
            county_data_pivot[race],
            width=bar_width,
            label=race,
            color=color
        )
    
    # Adjust text sizes
    plt.title(f"{county} Population Over Time (By Race/Ethnicity)", fontsize=25, pad=20)  # Larger title with padding
    plt.ylabel("Population Proportion", fontsize=20)  
    plt.xticks(
        [x + bar_width * (len(county_data_pivot.columns) - 1) / 2 for x in x_positions], 
        county_data_pivot.index, 
        rotation=30,  # Slightly tilt labels
        fontsize=18  # Larger tick labels
    )
    plt.yticks(fontsize=18)  # Larger y-axis tick labels
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add light grid lines for y-axis
    plt.legend(
        loc='upper center', 
        bbox_to_anchor=(0.5, -0.12),  # Add more padding below the chart for legend
        ncol=3,  # Limit to 3 columns for compactness
        fontsize=20,  # Larger legend text
        title_fontsize=16  # Larger legend title
    )
    
    # Save the plot as a PNG file and add to mapping
    plot_path = os.path.join(output_dir, f"{county}_population_over_time_adjacent_V2.png")
    plt.savefig(plot_path, bbox_inches='tight')  # Use bbox_inches to avoid cropping
    plt.close()
    
    # Placeholder ID for demonstration as upload to AGO isn't performed here
    county_item_mapping[county] = f"simulated_item_id_for_{county}"


# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)

mapping_file_path

'AGO_County_ItemID_Mapping.csv'

### Creating Excel sheet and viz for each region

In [98]:
# Load the uploaded file to review its content
file_path =REGIONAL_SUMMARY_DATA_LABELED_PATH

# Load the data from the CSV file
data = pd.read_csv(file_path)

# Create an Excel writer object
with pd.ExcelWriter('outputData/plots/county_line_charts.xlsx', engine='xlsxwriter') as writer:
    
    # Loop through each unique county and create a chart
    for county in data['COUNTY'].unique():
        county_data = data[data['COUNTY'] == county]
        
        
        # Ensure 'YEAR', 'POP', and 'RACE' columns are present
        county_data = county_data[['YEAR', 'POP_PROP', 'RACE']]
        
        # Aggregate to handle duplicates by taking the mean or sum
        county_data = county_data.groupby(['YEAR', 'RACE'], as_index=False).agg({'POP_PROP': 'mean'})
        
        # Pivot data to have 'RACE' as columns for plotting
        county_data_pivot = county_data.pivot(index='YEAR', columns='RACE', values='POP_PROP')
        
        # Create a line chart for the current county with a line for each RACE
        morpc.data_chart_to_excel(
            df=county_data_pivot,
            writer=writer,
            sheet_name=county,
            chartType='line',
            chartOptions={
                "titles": {
                    "chartTitle": f"{county} Population Over Time",
                    "xTitle": "Year",
                    "yTitle": "Population Proportion"
                },
                "legendOptions": {
                    "position": "bottom"
                }
            }
        )


### Uploading images to AGO

In [103]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Directory where images are saved
output_dir = 'outputData/plots'

# Initialize dictionary to store county-item mappings
county_item_mapping = {}

# Loop through each file in the directory and upload it
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "tags": "county, population, race, demographics",
            "description": f"Population proportion over time by race for {county_name} county.",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item


        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Store item ID in the dictionary
            county_item_mapping[county_name] = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{image_item.id}/data"
            
            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)



Deleted existing item with title: 15County population over time adjacent V2


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15County_population_over_time_adjacent_V2.png with item ID: 55860101439840ff8ae7daa8f056935e and set it to public
Deleted existing item with title: Delaware population over time adjacent V2


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_population_over_time_adjacent_V2.png with item ID: d2c5a23b0e544cb2a3296a6169f0c038 and set it to public
Deleted existing item with title: Fairfield population over time adjacent V2


...

Uploaded Pickaway_population_over_time_adjacent_V2.png with item ID: 1a8467e4e8454999a9d929839495eb8a and set it to public
Deleted existing item with title: Ross population over time adjacent V2


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_population_over_time_adjacent_V2.png with item ID: d3d2353c32a14ea7b0af5509098e7449 and set it to public
Deleted existing item with title: Union population over time adjacent V2


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_population_over_time_adjacent_V2.png with item ID: 0ca64b1e0f6e475fb589973b15aacf76 and set it to public


### Update Metadata Local

In [104]:
from arcgis.gis import GIS
import pandas as pd
import os

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Access the existing Excel item by its itemID and download it
item_id = "012c09a5e5344954a573a1677a441e78"
excel_item = gis.content.get(item_id)

# Download the existing Excel file
download_path = excel_item.download(save_path="inputData/")
existing_df = pd.read_excel(download_path)

# Load the mapping CSV file (created earlier with COUNTY to itemID mappings)
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df = pd.read_csv(mapping_file_path)


# Define a function to determine 'RegionType' based on 'COUNTY'
def determine_region_type(county):
    if county == '15County':
        return 'Region-Level'
    elif county in [
        'Delaware', 'Fairfield', 'Fayette', 'Franklin', 'Hocking', 'Knox',
        'Licking', 'Logan', 'Madison', 'Marion', 'Morrow', 'Perry', 
        'Pickaway', 'Ross', 'Union'
    ]:
        return 'County-Level'
    else:
        return 'Community-Level'

# Add new columns to mapping for 'Image', 'RegionType', and 'RegionName'
mapping_df['Image_'] = mapping_df['itemID']
mapping_df['RegionType'] = mapping_df['COUNTY'].apply(determine_region_type)
mapping_df['RegionName'] = mapping_df['COUNTY']

# Select the relevant columns from the mapping DataFrame to append to the existing Excel data
mapping_to_append = mapping_df[['Image_', 'RegionType', 'RegionName']]

# Append the new rows to the existing DataFrame
updated_df = pd.concat([existing_df, mapping_to_append], ignore_index=True)

# Save the updated DataFrame to a new Excel file
updated_file_path = 'outputData/Insights_Metadata.xlsx'
updated_df.to_excel(updated_file_path, index=False)

# Update the item with the new file
excel_item.update({}, updated_file_path)

# Confirm the update
print(f"Updated item {excel_item.title} with new data.")


C:\Users\ogwynn\newvirt\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


Updated item Insights_Metadata with new data.


In [93]:
mapping_df.head()

,COUNTY,itemID,Image_,RegionType,RegionName
0,15County,https://morpc.maps.arcgis.com/sharing/rest/con...,https://morpc.maps.arcgis.com/sharing/rest/con...,Region-Level,15County
1,Delaware,https://morpc.maps.arcgis.com/sharing/rest/con...,https://morpc.maps.arcgis.com/sharing/rest/con...,County-Level,Delaware
2,Fairfield,https://morpc.maps.arcgis.com/sharing/rest/con...,https://morpc.maps.arcgis.com/sharing/rest/con...,County-Level,Fairfield
3,Fayette,https://morpc.maps.arcgis.com/sharing/rest/con...,https://morpc.maps.arcgis.com/sharing/rest/con...,County-Level,Fayette
4,Hocking,https://morpc.maps.arcgis.com/sharing/rest/con...,https://morpc.maps.arcgis.com/sharing/rest/con...,County-Level,Hocking


### Updating Insights Metadata

In [101]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Access the hosted table layer
table_item = gis.content.get("5997df4c5bab4c3b947b2dfab95104be")

# Access the hosted table layer
if table_item.tables:
    table_layer = table_item.tables[0]  # Get the first (or only) table in the item
else:
    raise ValueError("No table found in the specified item.")
    
# Load the mapping CSV
mapping_df = pd.read_csv('AGO_County_ItemID_Mapping.csv')  # Adjust the path as needed

# Query existing records in the hosted table
existing_features = table_layer.query().features
existing_data = pd.DataFrame([feature.attributes for feature in existing_features])

# Prepare list to store updates
updates = []

print(mapping_df)


# Iterate through each row in the mapping CSV to check and prepare updates
for _, row in mapping_df.iterrows():
    county_name = row['COUNTY']
    item_id = row['itemID']
    
    # Determine the values for RegionType and RegionName
    if county_name == "15County":
        region_type = "Region-Level"
    elif county_name in {"Delaware", "Fairfield", "Fayette", "Franklin", "Hocking", "Knox", "Licking", 
                         "Logan", "Madison", "Marion", "Morrow", "Perry", "Pickaway", "Ross", "Union"}:
        region_type = "County-Level"
    else:
        region_type = "Community-Level"
    
    # Construct the Image URL
    image_url = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{item_id}/data"
    
    # Check if the Image already exists in the hosted table
    if (existing_data['Image_'] == image_url).any():
        continue  # Skip this row if the Image already exists

    # Find the feature to update or prepare a new one
    matching_features = existing_data[existing_data['RegionName'] == county_name]
    
    if not matching_features.empty:
        # Update existing feature
        feature_to_update = existing_features[matching_features.index[0]]
        feature_to_update.attributes.update({
            'Image_': image_url,  # Ensure no extra spaces in field names
            'RegionType': region_type,
            'RegionName': county_name
        })
        updates.append(feature_to_update)
    else:
        # Prepare a new feature if no match found
        new_feature = {
            'attributes': {
                'Image_': image_url,
                'RegionType': region_type,
                'RegionName': county_name
            }
        }
        updates.append(new_feature)

# Apply updates to the hosted table
if updates:
    # Convert features to dictionaries to avoid circular references
    updates_dict = [f.as_dict if hasattr(f, 'as_dict') else f for f in updates]
    table_layer.edit_features(updates=updates_dict)
    print("Updates applied to the hosted table on AGO.")
else:
    print("No updates needed. All entries are up-to-date.")


       COUNTY                                             itemID
0    15County  https://morpc.maps.arcgis.com/sharing/rest/con...
1    Delaware  https://morpc.maps.arcgis.com/sharing/rest/con...
2   Fairfield  https://morpc.maps.arcgis.com/sharing/rest/con...
3     Fayette  https://morpc.maps.arcgis.com/sharing/rest/con...
4    Franklin  https://morpc.maps.arcgis.com/sharing/rest/con...
5     Hocking  https://morpc.maps.arcgis.com/sharing/rest/con...
6        Knox  https://morpc.maps.arcgis.com/sharing/rest/con...
7     Licking  https://morpc.maps.arcgis.com/sharing/rest/con...
8       Logan  https://morpc.maps.arcgis.com/sharing/rest/con...
9     Madison  https://morpc.maps.arcgis.com/sharing/rest/con...
10     Marion  https://morpc.maps.arcgis.com/sharing/rest/con...
11     Morrow  https://morpc.maps.arcgis.com/sharing/rest/con...
12      Perry  https://morpc.maps.arcgis.com/sharing/rest/con...
13   Pickaway  https://morpc.maps.arcgis.com/sharing/rest/con...
14       Ross  https://mo